### Compute mean coherences and convert to DataFrame

In [1]:
import sys

sys.path.insert(1, "/home/vinicius/storage1/projects/GrayData-Analysis")

In [5]:
import os

from tqdm import tqdm
from frites.dataset import DatasetEphy
from frites.estimator import GCMIEstimator
from frites.workflow import WfMi
from config import sessions
import xarray as xr
from GDa.util import average_stages
from GDa.temporal_network import temporal_network

#### Iterate over all sessions and concatenate coherece

In [3]:
coh_file = f'coh_k_0.3_multitaper_at_cue.nc'
coh_sig_file = f'coh_k_0.3_multitaper_at_cue_surr.nc'

coh = []
for s_id in tqdm(sessions):
    net = temporal_network(coh_file=coh_file,
                           coh_sig_file=coh_sig_file, wt=None,
                           date=s_id, trial_type=[1],
                           behavioral_response=[1])
    # Average if needed
    out = average_stages(net.super_tensor, 1)
    # To save memory
    del net
    # Convert to format required by the MI workflow
    coh += [out.isel(roi=[r])
            for r in range(len(out['roi']))]

100%|████████████████████████████████████████| 62/62 [3:20:13<00:00, 193.77s/it]


In [ ]:
# Concatenate channels
data = xr.concat(coh, dim="roi")
# Get unique rois
urois, counts = np.unique(data.roi.data, return_counts=True)
# Get unique rois that has at leats 10 channels
urois = urois[counts >= 10]
# Average channels withn the same roi
data = data.groupby("roi").mean("roi", skipna=True)
data = data.sel(roi=urois)